Requirements

In [53]:
with open("requirements.txt", "w") as f:
    f.write("requests==2.31.0\n")  # Versão mais recente compatível com o código
    f.write("pandas==1.5.3\n")     # Versão compatível com a manipulação de dados
    f.write("sqlite3==3.8.3\n")    # Parte do Python, só para referência

print("Arquivo 'requirements.txt' criado com sucesso!")

Arquivo 'requirements.txt' criado com sucesso!


Bibliotecas

In [54]:
# Importar bibliotecas
import requests  # Para acessar a API
import pandas as pd  # Para manipular os dados
import sqlite3  # Para armazenar os dados em SQLite

Função para acessar a API e retornar os dados

In [55]:
def acessar_api(url):
    try:
        response = requests.get(url)  # Fazendo a requisição GET
        response.raise_for_status()  # Verificando se houve erro na requisição
        return response.json()  # Retorna os dados no formato JSON
    except requests.exceptions.RequestException as e:
        print(f"Erro ao acessar a API ({url}): {e}")
        return None

APIs

In [56]:
# URL das APIs
url_bancos = "https://brasilapi.com.br/api/banks/v1"
url_feriados = "https://brasilapi.com.br/api/feriados/v1/2024"
url_cnpjs = "https://brasilapi.com.br/api/cnpj/v1/00000000000191"

# Extrair dados das APIs
dados_bancos = acessar_api(url_bancos)
dados_feriados = acessar_api(url_feriados)
dados_cnpj = acessar_api(url_cnpjs)

# Verificar se os dados foram obtidos
if dados_bancos and dados_feriados and dados_cnpj:
    print("Dados de todas as APIs extraídos com sucesso!")
else:
    if not dados_bancos:
        print("⚠️ Alerta: Não foi possível obter os dados da API de Bancos.")
    if not dados_feriados:
        print("⚠️ Alerta: Não foi possível obter os dados da API de Feriados.")
    if not dados_cnpj:
        print("⚠️ Alerta: Não foi possível obter os dados da API de CNPJ.")

Dados de todas as APIs extraídos com sucesso!


Tabela 1: Dados de Bancos

In [57]:
if dados_bancos:
    df_bancos = pd.DataFrame(dados_bancos)
    df_bancos = df_bancos.drop_duplicates()  # Remover duplicatas, pois dados repetidos não são úteis
    df_bancos = df_bancos.fillna("Desconhecido")  # Preencher valores nulos com "Desconhecido", justificável para evitar dados incompletos
    df_bancos.columns = ['ISPB', 'Nome_Banco', 'Codigo_Banco', 'Nome_Completo']
    df_bancos['Nome_Banco'] = df_bancos['Nome_Banco'].str.upper()
    df_bancos['Nome_Completo'] = df_bancos['Nome_Completo'].str.upper()
    print("Tabela de Bancos criada com sucesso!")
else:
    df_bancos = pd.DataFrame()

Tabela de Bancos criada com sucesso!


Tabela 2: Dados de Feriados

In [60]:
if dados_feriados:
    df_feriados = pd.DataFrame(dados_feriados)
    if len(df_feriados.columns) == 2:  # Verificar se o número de colunas é o esperado
        df_feriados.columns = ['Data', 'Nome']
        print("Tabela de Feriados criada com sucesso!")
    else:
        print("⚠️ Formato inesperado dos dados de feriados.")
        display(df_feriados.head())
else:
    df_feriados = pd.DataFrame()

⚠️ Formato inesperado dos dados de feriados.


,date,name,type
0,2024-01-01,Confraternização mundial,national
1,2024-02-13,Carnaval,national
2,2024-03-29,Sexta-feira Santa,national
3,2024-03-31,Páscoa,national
4,2024-04-21,Tiradentes,national


Tabela 3: Dados de CNPJ

In [61]:
if dados_cnpj:
    df_cnpj = pd.json_normalize(dados_cnpj)  # Normalizar os dados JSON aninhados
    # Filtrar colunas relevantes para evitar redundâncias no banco de dados
    colunas_relevantes = ['cnpj', 'razao_social', 'descricao_situacao_cadastral']
    df_cnpj = df_cnpj[colunas_relevantes]
    # Preencher valores nulos com "Desconhecido", justificável para garantir integridade dos dados
    df_cnpj = df_cnpj.fillna("Desconhecido")
    display("Tabela de CNPJ criada com sucesso!")
else:
    df_cnpj = pd.DataFrame()

'Tabela de CNPJ criada com sucesso!'

Conectar e Salvar no Banco de Dados

In [62]:
# Conectar ao banco SQLite
conexao = sqlite3.connect("dados_multiplas_apis.db")

In [63]:
# Salvar as tabelas no banco de dados
if not df_bancos.empty:
    df_bancos.to_sql("tabela_bancos", conexao, if_exists="replace", index=False)
if not df_feriados.empty:
    df_feriados.to_sql("tabela_feriados", conexao, if_exists="replace", index=False)
if not df_cnpj.empty:
    df_cnpj.to_sql("tabela_cnpj", conexao, if_exists="replace", index=False)

Fechar Conexão

In [64]:
# Fechar conexão
conexao.close()
display("Todas as tabelas foram armazenadas com sucesso no banco SQLite 'dados_multiplas_apis.db'.")

"Todas as tabelas foram armazenadas com sucesso no banco SQLite 'dados_multiplas_apis.db'."

Validação

In [65]:
# Validar os dados exportados no banco SQLite
conexao = sqlite3.connect("dados_multiplas_apis.db")

In [66]:
# Verificar a tabela de Bancos
if not df_bancos.empty:
    bancos_exportados = pd.read_sql_query("SELECT * FROM tabela_bancos", conexao)
    display("Validação - Tabela de Bancos:")
    display(bancos_exportados.head())

'Validação - Tabela de Bancos:'

,ISPB,Nome_Banco,Codigo_Banco,Nome_Completo
0,00000000,BCO DO BRASIL S.A.,1.0,BANCO DO BRASIL S.A.
1,00000208,BRB - BCO DE BRASILIA S.A.,70.0,BRB - BANCO DE BRASILIA S.A.
2,00038121,SELIC,Desconhecido,BANCO CENTRAL DO BRASIL - SELIC
3,00038166,BACEN,Desconhecido,BANCO CENTRAL DO BRASIL
4,00122327,SANTINVEST S.A. - CFI,539.0,"SANTINVEST S.A. - CREDITO, FINANCIAMENTO E INV..."


In [67]:
# Verificar a tabela de Feriados
if not df_feriados.empty:
    feriados_exportados = pd.read_sql_query("SELECT * FROM tabela_feriados", conexao)
    display("Validação - Tabela de Feriados:")
    display(feriados_exportados.head())

'Validação - Tabela de Feriados:'

,date,name,type
0,2024-01-01,Confraternização mundial,national
1,2024-02-13,Carnaval,national
2,2024-03-29,Sexta-feira Santa,national
3,2024-03-31,Páscoa,national
4,2024-04-21,Tiradentes,national


In [68]:
# Verificar a tabela de CNPJ
if not df_cnpj.empty:
    cnpj_exportados = pd.read_sql_query("SELECT * FROM tabela_cnpj", conexao)
    display("Validação - Tabela de CNPJ:")
    display(cnpj_exportados.head())

'Validação - Tabela de CNPJ:'

,cnpj,razao_social,descricao_situacao_cadastral
0,00000000000191,BANCO DO BRASIL SA,ATIVA


Fechar Conexão

In [69]:
# Fechar conexão após validação
conexao.close()